In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import random
from sklearn.metrics.pairwise import cosine_similarity

TEST_SET_THRESHOLD = 10
TEST_SET_HOLDOUT = 0.2

In [3]:
tracks = pd.read_csv('../input/tracks.csv')
train = pd.read_csv('../input/train.csv')
target = pd.read_csv('../input/target_playlists.csv')

# Defining methods to create csr matrices

In [4]:
def classify_durations(data):
    data.loc[tracks['duration_sec'].isin(range(60)),'duration_sec'] = 1
    data.loc[tracks['duration_sec'].isin(range(60,120)), 'duration_sec'] = 2
    data.loc[tracks['duration_sec'].isin(range(120,180)), 'duration_sec'] = 3
    data.loc[tracks['duration_sec'].isin(range(180,240)), 'duration_sec'] = 4
    data.loc[tracks['duration_sec'].isin(range(240,300)), 'duration_sec'] = 5
    data.loc[tracks['duration_sec'].isin(range(300,200000)), 'duration_sec'] = 6

def build_urm_csr(data):
    fill_data = np.ones(data.shape[0])
    #posso usare gli id direttamente solo perchè come già detto sono consistenti
    row = data['playlist_id'].values
    col = data['track_id'].values
    n_pl = np.amax(data['playlist_id']) + 1
    n_tr = np.amax(data['track_id']) + 1
    
    return sparse.csr_matrix((fill_data, (row, col)), dtype=np.int32, shape=(n_pl, n_tr))

def build_icm_csr(data):
    
    classify_durations(data)
    
    albums_id = data['album_id']
    artists_id = data['artist_id']
    durations = data['duration_sec']
    tracks = data['track_id']
    
    albums_max = np.amax(albums_id)
    artists_max = np.amax(artists_id)
    durations_max = np.amax(durations)
    number_of_songs = data.shape[0]
    
    icm_csr_matrix = sparse.csr_matrix((number_of_songs, albums_max + artists_max + durations_max + 3), dtype=np.uint32)
    
    icm_csr_matrix[tracks,albums_id] = 1
    icm_csr_matrix[tracks, albums_max + artists_id] = 1
    icm_csr_matrix[tracks, albums_max + artists_max + durations] = 1

    return icm_csr_matrix

# SPLITTING DATASET

In [5]:
#Raggruppiamo per playlist_id, le celle conterranno il count() del gruppo, quindi il numero di canzoni per playlist
grouped = train.groupby('playlist_id')['track_id'].nunique()

#Prendiamo le playlist che superano il numero di elementi del TEST_SET_THRESHOLD
clipped = grouped.index[grouped>TEST_SET_THRESHOLD].tolist()

#Adesso prendiamo a caso degli indici di playlist in percentuale di TEST_SET_HOLDOUT
#ATTENZIONE, la percentuale viene calcolata sulla lunghezza di clipped, che avrà un numero di elementi inferiore a train
#Questo significa che il 20% di clipped sarà circa il 14% del train, la percentuale è da aggiustare tenendo conto di sto fatto
test_set_indices = [ clipped[i] for i in sorted(random.sample(range(len(clipped)), int(TEST_SET_HOLDOUT*len(clipped)))) ]

#Andiamo a estrarre dal train TUTTE le canzoni delle playlist estratte a sorte nella riga prima
test_groups = train.loc[train['playlist_id'].isin(test_set_indices)]

#Andiamo a creare un dataframe vuoto, a cui appenderemo tutte le canzoni da ficcare nel test_set con una .append()
test_set = pd.DataFrame(columns=["playlist_id","track_id"])

#Per ogni gruppo prendiamo le ultime 10 canzoni e le appendiamo al test_set
for name, group in test_groups.groupby('playlist_id'):
    test_set = test_set.append(group.tail(10))
    
#Togliamo le canzoni del test set al train, salvandolo in una nuova variabile 
#Questo è solo un trick per fare la differenza insiemistica
training_set = pd.concat([train, test_set, test_set]).drop_duplicates(keep=False)

#Ora passiamo training_set e test_set a csr_matrix
urm_csr = build_urm_csr(training_set)
test_set_csr = build_urm_csr(test_set)
icm_csr = build_icm_csr(tracks)

test_set_playlists = test_set['playlist_id'].unique()

C:\Users\giuse\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


# EVALUATION METHODS

In [11]:
def precision(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

def recall(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

def MAP(recommended_items, relevant_items):
       
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score


def evaluate_algorithm(URM_test, recommender_object, target_playlists, at=10):
    
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0
    
    num_eval = 0


    result = []
    
    for user_id in target_playlists:
    
        target_items = URM_test.getrow(user_id).indices
        
        recommended_items = recommender_object.recommend(user_id, at=at)
        num_eval+=1
        
        cumulative_precision += precision(recommended_items, target_items)
        cumulative_recall += recall(recommended_items, target_items)
        cumulative_MAP += MAP(recommended_items, target_items)
        
        recommendation_string = " ".join(str(i) for i in recommended_items)
        temp = [user_id,recommendation_string]
        result.append(temp)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval
    
    rec = pd.DataFrame(result)
    rec.to_csv("sample_submission.csv", index = False, header = ["playlist_id", "track_ids"])
    
    print("Recommender performance is: Precision = {:.6f}, Recall = {:.6f}, MAP = {:.6f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP)) 

# ALGORITHM

In [7]:
class TopPopRecommender(object):

    def fit(self, train):
        
        self.train = train

        itemPopularity = (train>0).sum(axis=0)
        itemPopularity = np.array(itemPopularity).squeeze()

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popularItems = np.argsort(itemPopularity)
        self.popularItems = np.flip(self.popularItems, axis = 0)
    
    
    def recommend(self, user_id, at=10, remove_seen=True):

        if remove_seen:
            unseen_items_mask = np.in1d(self.popularItems, self.train[user_id].indices,
                                        assume_unique=True, invert = True)

            unseen_items = self.popularItems[unseen_items_mask]

            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popularItems[0:at]
            
        
        return recommended_items

In [8]:
class ContentBasedRecommender(object):

    def fit(self, urm, icm):
        self.top_recommender = TopPopRecommender()
        self.top_recommender.fit(urm)
        
        self.similarities = cosine_similarity(icm, dense_output = False)
        self.urm = urm
        self.icm = icm
    
    def recommend(self, playlist_id, at=10, remove_seen = True):
        playlist = self.urm.getrow(playlist_id)
        if(len(playlist.data) < 0) :
            recommended_items = self.top_recommender.recommend(playlist_id)
        else:
            item_popularity = np.array(playlist.dot(self.similarities).todense())[0]
            self.popularitems = np.argsort(item_popularity)
            self.popularitems = np.flip(self.popularitems, axis = 0)

            if remove_seen:
                unseen_items_mask = np.in1d(self.popularitems, self.urm[playlist_id].indices,
                                            assume_unique=True, invert = True)

                unseen_items = self.popularitems[unseen_items_mask]

                recommended_items = unseen_items[0:at]

            else:
                recommended_items = self.popularitems[0:at]
            
        return recommended_items
        

# Testing algorithm

In [12]:
content_based_recommender = ContentBasedRecommender()
content_based_recommender.fit(urm_csr, icm_csr)

evaluate_algorithm(test_set_csr, content_based_recommender, test_set_playlists)


Recommender performance is: Precision = 0.062137, Recall = 0.062137, MAP = 0.034456


In [ ]:
urm_csr.getrow(0).indices

In [17]:
similarities = icm_csr.dot(icm_csr.transpose())

In [27]:
playlist = urm_csr.getrow(0)
item_popularity = np.array(playlist.dot(similarities).todense())[0]
popularitems = np.argsort(item_popularity)
popularitems = np.flip(popularitems)

In [28]:
playlist.shape

(1, 20635)

In [17]:
playlist.size

5